# Test Symbolics.jl

In [28]:
using Symbolics
function lotka_volterra!(du, u, p, t)
  x, y = u
  α, β, δ, γ = p
  du[1] = dx = α*x - β*x*y
  du[2] = dy = -δ*y + γ*x*y
end

lotka_volterra! (generic function with 1 method)

In [29]:
@variables t du[1:2] u[1:2] p[1:4]
du = collect(du)
lotka_volterra!(du, u, p, t)
du


2-element Vector{Num}:
 p[1]*u[1] - p[2]*u[1]*u[2]
 p[4]*u[1]*u[2] - p[3]*u[2]

In [52]:
du0 = rand(2)
A0 = rand(2, 2)
H0 = rand(2, 2, 2)
u0 = rand(2)
p0 = rand(4)
t0 = rand()

0.5700935946618089

In [53]:
f = build_function(du, u, p, t, target=Symbolics.CTarget(), expression=Val{false})

RuntimeGeneratedFunction(#=in Symbolics=#, #=using Symbolics=#, :((du, u, p, t)->begin
          #= /Users/astomodynamics/.julia/packages/Symbolics/3jLt1/src/build_function.jl:775 =#
          ccall(("diffeqf", "/var/folders/t6/s10dx_c10zsfxr8ymgs3nnv80000gn/T/jl_zALgLwBQbq"), Cvoid, (Ptr{Float64}, Ptr{Float64}, Ptr{Float64}, Float64), du, u, p, t)
      end))

In [57]:
dx = rand(2)
println(dx)
f(dx, u0, p0, t0)
dx

[0.5072773446876787, 0.7480898183279452]


2-element Vector{Float64}:
  0.06006955983929846
 -0.2007612442040133

In [56]:
[p0[1] * u0[1] - p0[2] * u0[1] * u0[2]
-p0[3] * u0[2] + p0[4] * u0[1] * u0[2]]

2-element Vector{Float64}:
  0.06006955983929846
 -0.2007612442040133

In [59]:
# ∇f = Symbolics.jacobian((u) -> f(du, u, p, t), u)
∇f = Symbolics.jacobian(du, u)


2×2 Matrix{Num}:
 p[1] - p[2]*u[2]        -p[2]*u[1]
        p[4]*u[2]  p[4]*u[1] - p[3]

In [60]:
∇f = Symbolics.build_function(∇f, u, p, t, target=Symbolics.CTarget(), expression=Val{false})

RuntimeGeneratedFunction(#=in Symbolics=#, #=using Symbolics=#, :((du, u, p, t)->begin
          #= /Users/astomodynamics/.julia/packages/Symbolics/3jLt1/src/build_function.jl:775 =#
          ccall(("diffeqf", "/var/folders/t6/s10dx_c10zsfxr8ymgs3nnv80000gn/T/jl_rSXSvozq7S"), Cvoid, (Ptr{Float64}, Ptr{Float64}, Ptr{Float64}, Float64), du, u, p, t)
      end))

In [61]:
A = [1.0 2.0; 1.2 0.1]
println(A)
∇f(A, u0, p0, t0)
A

[1.0 2.0; 1.2 0.1]


2×2 Matrix{Float64}:
 0.126007   -0.0636915
 0.0739126  -0.77123

In [66]:
[
    -p0[2] * u0[2] + p0[1]    -p0[2] * u0[1]
    p0[4] * u0[2]   -p0[3] + p0[4] * u0[1]]

2×2 Matrix{Float64}:
 0.126007   -0.0636915
 0.0739126  -0.77123

In [67]:
∇f = Symbolics.jacobian(du, u)
∇2f = Symbolics.jacobian(∇f, u)

4×2 Matrix{Num}:
     0  -p[2]
     0   p[4]
 -p[2]      0
  p[4]      0

In [68]:

# ∇2f = reshape(∇2f, 2, 2, 2)
∇2f = Symbolics.build_function(∇2f, u, p, t, target=Symbolics.CTarget(), expression=Val{false})

H = rand(2, 2, 2)
println(H)
∇2f(H, u0, p0, t0)
# rand(4)
H

[0.9359915116573716 0.4682478982122513; 0.8506274527032879 0.7351350264010803;;; 0.9996746966754817 0.7516198606332631; 0.06225947232277529 0.9401037879482768]


2×2×2 Array{Float64, 3}:
[:, :, 1] =
 0.0  -0.133604
 0.0   0.283937

[:, :, 2] =
 -0.133604  0.0
  0.283937  0.0

In [70]:
[0  -p0[2];
0 p0[4] ; -p0[2] 0; p0[4] 0]

4×2 Matrix{Float64}:
  0.0       -0.133604
  0.0        0.283937
 -0.133604   0.0
  0.283937   0.0

In [20]:
println(∇2f(randn(2),rand(2), rand(4), randn()))

nothing


In [5]:
build_function(du, u, p, t, target=Symbolics.CTarget())

"#include <math.h>\nvoid diffeqf(double* du, const double* RHS1, const double* RHS2, const double RHS3) {\n  du[0] = RHS2[0] * RHS1[0] + -1 * RHS2[1] * RHS1[0] * RHS1[1];\n  du[1] = -1 * RHS2[2] * RHS1[1] + RHS2[3] * RHS1[0] * RHS1[1];\n}\n"

In [6]:
f = build_function(du, u, p, t, target=Symbolics.CTarget(), expression=Val{false})

RuntimeGeneratedFunction(#=in Symbolics=#, #=using Symbolics=#, :((du, u, p, t)->begin
          #= /Users/astomodynamics/.julia/packages/Symbolics/3jLt1/src/build_function.jl:775 =#
          ccall(("diffeqf", "/var/folders/t6/s10dx_c10zsfxr8ymgs3nnv80000gn/T/jl_pyHaRGJQBa"), Cvoid, (Ptr{Float64}, Ptr{Float64}, Ptr{Float64}, Float64), du, u, p, t)
      end))

In [7]:
du = rand(2); du2 = rand(2)
u = rand(2)
p = rand(4)
t = rand()
f(du, u, p, t)
lotka_volterra!(du2, u, p, t)
du == du2 # true!

true

UndefVarError: UndefVarError: `jacobian!` not defined